# client instance 생성

In [1]:
DEPLOYMENT_URL="http://10.8.1.229:8000/"
# DEPLOYMENT_URL="http://localhost:2024/"
# DEPLOYMENT_URL="http://localhost:8000/"

from langgraph_sdk import get_client

client = get_client(url=DEPLOYMENT_URL)

# thread 생성 (앨런의 channel에 대응)

- 생성 요청 후 thread_id를 포함한 데이터를 return 받음
- 하나의 thread에 대해서 여러 에이전트를 호출하고 대화가능
- 현재 앨런은 한개의 channel에 대해서 한개의 agent가 1:1로 대응되므로 현재는 고려할 필요 없지만, 추후에 1개의 채널에 대해서 여러 agent를 호출가능

In [2]:
def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data
 
 
def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)
 
        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    if msg_type == "ai":
                        return True
    return False

In [3]:
thread = await client.threads.create()
print(thread)

thread_id = thread["thread_id"]
print(thread_id)

{'thread_id': '5d82d2d8-862a-434d-903b-05d567a3f2b1', 'created_at': '2025-08-18T00:51:22.194002+00:00', 'updated_at': '2025-08-18T00:51:22.194002+00:00', 'metadata': {}, 'status': 'idle', 'config': {}, 'values': None, 'interrupts': {}, 'error': None}
5d82d2d8-862a-434d-903b-05d567a3f2b1


# 에이전트 호출

- 해당 thread에서 어떤 agent를 호출할 지 assistant_id를 포함하여 요청을 보냄

## parameters
- thread_id : 위에서 생성한 thread_id
- assistant_id : 앨런의 persona_id에 대응, agent를 구분하는 id, 슬라이드 에이전트의 경우 "slide_generate_agent"
- input : 해당 agent 요청에 대한 parameter, 현재는 messages만 존재, 앨런 페이지에서 유저로 부터 받는 input을 해당 parameter에 dict로 넣어줌
- stream_mode : 어떤 event에 따라서 메시지를 받을지 지정, updates(한 단계가 끝날 때마나 메시지가 옴), events(llm이 생성하는 단위로 메시지 생성)

In [4]:
list_msg_id = list()


def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data


def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)

        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    msg_id = data['messages'][-1]['id']
                    if msg_id not in list_msg_id and msg_type == "ai":
                        list_msg_id.append(msg_id)
                        return True
    return False


def filter_silde_data(chunk):
    # slide 관련 데이터를 담은 메시지를 필터링하는 함수
    # 슬라이드 한페이지 단위로 메시지가 옴
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'executor' in node_name:
            return True

In [5]:
assistant_id = "slide_generate_agent"
list_chunk = list()

In [6]:
input = { "messages": "제주도 여행"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    list_chunk.append(chunk)
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])
    # elif filter_silde_data(chunk):
    #     print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
    #     print(chunk)


##########  AI 메시지  ##########
제주도 여행을 주제로 슬라이드를 생성하도록 하겠습니다.
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. 제주도 추천 여행지 - 이 섹션에서는 제주도의 대표적인 여행 명소와 해당 명소의 매력을 소개합니다. 성산일출봉, 서귀포의 중문관광단지, 협재해수욕장 등 주목할 만한 명소를 포함하며, 각 지역의 대표적인 관광지의 매력을 살펴봅니다. 
3. 제주도의 맛 - 제주도에서 시도할 만한 대표 음식을 소개합니다. 흑돼지, 갈치조림, 오메기떡 등 독특한 제주도 미식 체험에 대한 정보를 제공합니다. 
4. 제주도 교통 및 여행 팁 - 제주도 여행 시 유용한 교통 정보와 여행 팁을 공유합니다. 대중교통 이용 방법, 렌터카 추천, 교통카드 사용법 등을 다룹니다. 
5. 제주도 숙박 시설 정보 - 제주도의 다양한 숙박 옵션에 대해 소개합니다. 호텔, 펜션, 게스트하우스 등 다양한 선택지와 그 특징 및 가격 범위를 설명합니다. 
6. 제주도 여행 일정 및 코스 - 효율적인 제주도 여행을 위한 일정을 제안합니다. 2박 3일, 3박 4일 등 기간별 추천 일정을 포함하여 여행객의 선호에 맞춘 코스를 제공합니다. 

##########  AI 메시지  ##########

    슬라이드 생성에 필요한 조사를 시작합니다.
    
##########  AI 메시지  ##########

    슬라이드에 사용할 이미지를 검색하고 있습니다.
    
##########  AI 메시지  ##########

    슬라이드 디자인을 시작합니다.
    
##########  AI 메시지  ##########

    슬라이드를 생성하고 있습니다.
    
##########  AI 메시지  ##########
슬라이드 생성이 완료되었습니다.


In [7]:
for chunk in list_chunk:
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])


In [8]:
state = await client.threads.get_state(thread_id=thread_id)

for msg in state["values"]["messages"]:
    if msg["type"] != "human":
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        print(msg["content"])

##########  AI 메시지  ##########
제주도 여행을 주제로 슬라이드를 생성하도록 하겠습니다.
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. 제주도 추천 여행지 - 이 섹션에서는 제주도의 대표적인 여행 명소와 해당 명소의 매력을 소개합니다. 성산일출봉, 서귀포의 중문관광단지, 협재해수욕장 등 주목할 만한 명소를 포함하며, 각 지역의 대표적인 관광지의 매력을 살펴봅니다. 
3. 제주도의 맛 - 제주도에서 시도할 만한 대표 음식을 소개합니다. 흑돼지, 갈치조림, 오메기떡 등 독특한 제주도 미식 체험에 대한 정보를 제공합니다. 
4. 제주도 교통 및 여행 팁 - 제주도 여행 시 유용한 교통 정보와 여행 팁을 공유합니다. 대중교통 이용 방법, 렌터카 추천, 교통카드 사용법 등을 다룹니다. 
5. 제주도 숙박 시설 정보 - 제주도의 다양한 숙박 옵션에 대해 소개합니다. 호텔, 펜션, 게스트하우스 등 다양한 선택지와 그 특징 및 가격 범위를 설명합니다. 
6. 제주도 여행 일정 및 코스 - 효율적인 제주도 여행을 위한 일정을 제안합니다. 2박 3일, 3박 4일 등 기간별 추천 일정을 포함하여 여행객의 선호에 맞춘 코스를 제공합니다. 

##########  AI 메시지  ##########
슬라이드 생성이 완료되었습니다.


In [9]:
len(state["values"]["slides"])

7

In [10]:
state["values"]["slides"]

[{'slide_type': 'title',
  'topic': '제주도 여행',
  'idx': 0,
  'name': '타이틀',
  'description': '제주도 여행에 대한 프레젠테이션 타이틀 페이지',
  'requirements': ['타이틀 텍스트', '부제목 또는 설명'],
  'research': True,
  'content': '## 타이틀\n\n제주도는 한국을 대표하는 관광 명소로, 방문객들에게 다양하고 독특한 경험을 제공합니다. 제주도 여행 프레젠테이션의 타이틀 페이지는 여행객들이 떠날 여행의 첫인상을 형성하는 데 중요한 역할을 합니다. 이러한 타이틀 페이지는 제주도의 자연경관과 문화적 요소를 잘 반영해야 합니다.\n\n먼저, 제주도의 상징인 돌하르방이나 한라산을 배경 이미지로 선택하는 것이 좋습니다. 이 외에도 제주 특산물인 유채꽃과 감귤, 그리고 제주 해녀 문화를 엿볼 수 있는 이미지들이 타이틀 페이지를 더욱 풍부하게 만들어 줄 것입니다 [1][2]. 타이틀은 간단하면서도 인상적인 문구로 구성되며, 예를 들어 "신비의 섬, 제주로 떠나는 여정"과 같은 표현은 여행의 기대감을 높일 수 있습니다.\n\n디자인 측면에서는 깔끔하고 단정한 레이아웃을 활용하는 것이 좋습니다. 예를 들어, 미리캔버스에서 제공하는 제주도 여행 템플릿을 참고하여 심플하면서도 현대적인 스타일의 페이지를 구성하는 것이 좋습니다 [3]. 색상 선택에서는 제주도의 주요 색조인 파란색과 초록색을 사용하여 시각적으로 제주도의 자연과 맞물리도록 디자인합니다.\n\n타이틀 페이지는 여행의 전반적인 정보를 간단히 소개할 수 있는 서문 역할도 합니다. 여행의 목표, 주요 여행지, 및 참가자 정보를 간단히 요약하여 참석자들이 여행 계획의 개요를 쉽게 파악할 수 있도록 합니다.\n\n이렇게 설계된 타이틀 페이지는 청중에게 제주도의 매력을 효과적으로 전달하며, 성공적인 프레젠테이션의 시작점을 제공합니다.\n\n### Sources\n1. [제주도 여행 가이드 - Naver Blog

In [11]:
for s in state["values"]["slides"]:
    with open(f"{s['idx']}.html", "w", encoding="utf-8") as f:
        f.write(s['html'])